<a href="https://colab.research.google.com/github/KChamroeun/Application/blob/main/CalendarRender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

heavenly_stem_characters = {
    "Jia": "甲 ឈើ+",
    "Yi": "乙 ឈើ-",
    "Bing": "丙 ភ្លើង+",
    "Ding": "丁 ភ្លើង-",
    "Wu": "戊 ដី+",
    "Ji": "己 ដី-",
    "Geng": "庚 មាស+",
    "Xin": "辛 មាស-",
    "Ren": "壬 ទឹក+",
    "Gui": "癸 ទឹក-"
      }
heavenly_stem_characters_for_year = {
    0: "庚 Geng មាស+",
    1: "辛 Xin 庚 មាស-",
    2: "壬 Ren ទឹក+",
    3: "癸 Gui ទឹក-",
    4: "甲 Jia ឈើ+",
    5: "乙 Yi ឈើ-",
    6: "丙 Bing ភ្លើង+",
    7: "丁 Ding ភ្លើង-",
    8: "戊 Wu ដី+",
    9: "己 Ji ដី-"
}

print(heavenly_stem_characters_for_year)

print(heavenly_stem_characters)





{'Jia': '甲 ឈើ+', 'Yi': '乙 ឈើ-', 'Bing': '丙 ភ្លើង+', 'Ding': '丁 ភ្លើង-', 'Wu': '戊 ដី+', 'Ji': '己 ដី-', 'Geng': '庚 មាស+', 'Xin': '辛 មាស-', 'Ren': '壬 ទឹក+', 'Gui': '癸 ទឹក-'}


Render Calendar based on Lichun and Winter solstice

In [5]:
!pip install astronomy
!pip install Astronomy

In [6]:
# bazi_pillars.py  ──

from datetime import datetime, date, timezone, timedelta
from dataclasses import dataclass
from abc import ABC, abstractmethod
from astronomy import Astronomy, Body, Direction

# ----------------- constants -----------------
STEMS   = '甲乙丙丁戊己庚辛壬癸'
BRANCH  = '子丑寅卯辰巳午未申酉戌亥'
STEM_GROUP = {c: i//2 for i, c in enumerate(STEMS)}     # 甲/己→0 …

# Month‑stem lookup rows (5) × cols (12)
MONTH_STEMS = [
    list('丙丁戊己庚辛壬癸甲乙丙丁'),
    list('戊己庚辛壬癸甲乙丙丁戊己'),
    list('庚辛壬癸甲乙丙丁戊己庚辛'),
    list('壬癸甲乙丙丁戊己庚辛壬癸'),
    list('甲乙丙丁戊己庚辛壬癸甲乙'),
]

EPOCH_JZ = date(1984, 2, 2)                       # known 甲子 day

ASTRO = Astronomy()
TZ_UTC7 = timezone(timedelta(hours=7))            # Phnom Penh

# -------------- helpers -----------------
def julian_day(g: date) -> int:
    a = (14 - g.month)//12
    y = g.year + 4800 - a
    m = g.month + 12*a - 3
    return g.day + (153*m+2)//5 + 365*y + y//4 - y//100 + y//400 - 32045

def year_stem_branch(year:int) -> tuple[str,str]:
    stem = STEMS[(year - 1984) % 10]
    branch=BRANCH[(year - 1984) % 12]
    return stem, branch

def month_stem_branch(year_stem:str, solar_month:int) -> tuple[str,str]:
    stem = MONTH_STEMS[STEM_GROUP[year_stem]][solar_month-1]
    branch = BRANCH[(solar_month+1) % 12]   # Month 1 = 寅
    return stem, branch

def day_stem_branch(d: date) -> tuple[str,str]:
    delta = (julian_day(d) - julian_day(EPOCH_JZ)) % 60
    return STEMS[delta % 10], BRANCH[delta % 12]

def hour_branch(dt_local: datetime) -> str:
    h = dt_local.hour
    return BRANCH[((h+1)%24)//2]

def hour_stem(day_stem:str, h_branch:str) -> str:
    d_idx = STEMS.index(day_stem)
    b_idx = BRANCH.index(h_branch)
    return STEMS[(2*b_idx + d_idx) % 10]

# -------------- Strategy Pattern --------------
class Boundary(ABC):
    @abstractmethod
    def pivot(self, year:int) -> date: ...
    def bazi_year(self, d: date) -> int:
        return d.year if d >= self.pivot(d.year) else d.year-1
    def solar_month(self,d:date)->int:
        ref=self.pivot(d.year)
        if d<ref: ref=self.pivot(d.year-1)
        return 1+(d-ref).days//30            # rough but sufficient for pillar math

class LiChunBoundary(Boundary):
    name="LiChun"
    def pivot(self,year):
        evt=ASTRO.search_hour_angle(Body.sun,315,date(year,2,2),Direction.next)
        return evt.instant.to_utc_datetime().astimezone(TZ_UTC7).date()

class SolsticeBoundary(Boundary):
    name="Solstice"
    def pivot(self,year):
        evt=ASTRO.search_hour_angle(Body.sun,270,date(year,12,19),Direction.next)
        return evt.instant.to_utc_datetime().astimezone(TZ_UTC7).date()

BOUNDARY_MAP={"LiChun":LiChunBoundary(),"Solstice":SolsticeBoundary()}

# -------------- Public API -----------------
@dataclass
class Pillars:
    year:str; month:str; day:str; hour:str

def four_pillars(dt_local: datetime, mode:str="LiChun")->Pillars:
    strat=BOUNDARY_MAP[mode]
    byear = strat.bazi_year(dt_local.date())
    ystem,ybranch = year_stem_branch(byear)

    smonth = strat.solar_month(dt_local.date())
    mstem,mbranch = month_stem_branch(ystem,smonth)

    dstem,dbranch = day_stem_branch(dt_local.date())

    hbranch = hour_branch(dt_local)
    hstem   = hour_stem(dstem,hbranch)

    return Pillars(
        year = f"{ystem}{ybranch}",
        month= f"{mstem}{mbranch}",
        day  = f"{dstem}{dbranch}",
        hour = f"{hstem}{hbranch}",
    )

# ---------------- demo ----------------------
if __name__ == "__main__":
    born = datetime(2026,4,5,10,43,tzinfo=TZ_UTC7)
    for mode in ("LiChun","Solstice"):
        p = four_pillars(born,mode)
        print(mode, "→", p)


ImportError: cannot import name 'Astronomy' from 'astronomy' (/usr/local/lib/python3.11/dist-packages/astronomy/__init__.py)

Render Calendar

In [ ]:
import calendar

heavenly_stems = ["Jia", "Yi", "Bing", "Ding", "Wu", "Ji", "Geng", "Xin", "Ren", "Gui"]
earthly_branches = ["Zi", "Chou", "Yin", "Mao", "Chen", "Si", "Wu", "Wei", "Shen", "You", "Xu", "Hai"]
solar_terms_2025 = {  # Example - you'd need the actual dates
    "Start of Spring": "2025-02-03",
    "Rain Water": "2025-02-18",
    # ... and so on for all 24 solar terms
}

def get_day_pillar(year, month, day):
    # This is a simplified placeholder - the actual calculation is complex
    # You'd need a reference date and the 60 Jia Zi cycle logic
    index = (year + month + day) % 60
    heavenly_stem_index = index % 10
    earthly_branch_index = index % 12
    return heavenly_stems[heavenly_stem_index], earthly_branches[earthly_branch_index]

def get_month_pillar(year_heavenly_stem, month):
    # This is also a simplified placeholder - the logic is based on year stem
    # and the fixed Earthly Branch for the month
    year_stem_index = heavenly_stems.index(year_heavenly_stem)
    month_heavenly_stems = ["Bing", "Ding", "Wu", "Ji", "Geng", "Xin", "Ren", "Gui", "Jia", "Yi", "Bing", "Ding"] # Example sequence
    return month_heavenly_stems[(year_stem_index % 5) * 2 + (month - 1) % 12], earthly_branches[month - 1] # Assuming month starts from Yin

def render_bazi_calendar(year):
    cal = calendar.TextCalendar()
    year_stem = get_heavenly_stem_for_year(year) # Assuming you have this function

    for month in range(1, 13): # Looping through months (though Bazi has a different lunar calendar start)
        print(f"\n----- Month {month} -----")
        month_stem, month_branch = get_month_pillar(year_stem, month)
        print(f"Month Pillar: {month_stem} {month_branch}")
        print(cal.formatmonth(year, month))
        for day in range(1, calendar.monthrange(year, month)[1] + 1):
            day_stem, day_branch = get_day_pillar(year, month, day)
            date_str = f"{year}-{month:02d}-{day:02d}"
            solar_term = ""
            for term, date in solar_terms_2025.items(): # This needs to be for the correct year
                if date == date_str:
                    solar_term = f"({term})"
                    break
            print(f"{day}: {day_stem} {day_branch} {solar_term}")

# Example call (will not be fully accurate without the complex logic)
render_bazi_calendar(2025)

Winter Solstic on 21 or 22nd using skyfield library

In [ ]:
!pip install skyfield

from skyfield.api import load
from skyfield import almanac
from datetime import timezone

ts = load.timescale()
eph = load('de440s.bsp')

def december_solstice(year):
    t0 = ts.utc(year, 12, 19)
    t1 = ts.utc(year, 12, 24)
    times, events = almanac.find_discrete(t0, t1, almanac.seasons(eph))
    for t, e in zip(times, events):
        if e == 3:  # 3 means December solstice
            return t.utc_datetime().replace(tzinfo=timezone.utc)

print(december_solstice(2023))
